

<div class="alert alert-success">
    <h2 align='center'>⚙️ Downloads, Imports ⚙️</h2>
</div>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/RZDHack/train_dataset.zip -d data

!mv data/dataset.json data/

Archive:  /content/drive/MyDrive/RZDHack/train_dataset.zip
  inflating: data/dataset.json       
  inflating: data/readme.md          
mv: 'data/dataset.json' and 'data/dataset.json' are the same file


In [ ]:
!pip3 install torch torchaudio torchvision torchtext torchdata

In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 11.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch.autograd import Variable
from torch import nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder
import torch.nn.functional as F
import implicit

<div class="alert alert-success">
    <h2 align='center'>Metrics</h2>
</div>

In [ ]:
def calculate_precision_at_k(actual, predicted, k):
    predicted = predicted[:k]

    common_elements = set(predicted).intersection(set(actual))
    precision = len(common_elements) / k if k > 0 else 0.0

    return precision

def calculate_recall_at_k(actual, predicted, k):
    predicted = predicted[:k]
    common_elements = set(predicted).intersection(set(actual))
    recall = len(common_elements) / len(actual) if len(actual) > 0 else 0.0

    return recall

def calculate_average_precision_at_k(actual, predicted, k):
    sum_precision_at_k = 0.0

    for i in range(len(actual)):
        precision_at_k = calculate_precision_at_k(actual[i], predicted[i], k)
        sum_precision_at_k += precision_at_k
    average_precision_at_k = sum_precision_at_k / len(actual)

    return average_precision_at_k

def get_top_k_recommendations(predictions, k):
    top_k_indices = torch.topk(predictions, k).indices
    return top_k_indices

In [ ]:
def convert_duration_to_seconds(duration_str):
    parts = duration_str.split(':')
    hours, minutes, seconds = map(int, parts)
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

<div class="alert alert-success">
    <h2 align='center'>🍪 Cook the datas 🍪</h2>
</div>

In [ ]:
dataset_path = "/content/data/dataset.json"

In [ ]:
with open(dataset_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

In [ ]:
stations_data = [item['stations'] for item in data]
combined_data = {}

for item in stations_data:
    combined_data.update(item)

stations_df = pd.DataFrame.from_dict(combined_data, orient='index', columns=[f'station_{i}' for i in range(1, 8)], dtype="float64")
stations_df.index.name = 'station_id'
stations_df.reset_index(inplace=True)

In [ ]:
def extract_station_number(station_str):
    parts = station_str.split('(')
    if len(parts) == 2:
        number = parts[1].strip(')')
        return int(number)
    else:
        return None

stations_df['station_number'] = stations_df['station_id'].apply(extract_station_number)
stations_df.drop(columns=['station_id'], inplace=True)

In [ ]:
print("DataFrame для станций:")
stations_df.head()

DataFrame для станций:


,station_1,station_2,station_3,station_4,station_5,station_6,station_7,station_number
0,0.0,8.0,12.0,0.0,35.0,32.0,14.0,1
1,26.0,0.0,27.0,36.0,27.0,3.0,18.0,2
2,35.0,30.0,0.0,32.0,1.0,36.0,26.0,3
3,13.0,2.0,15.0,0.0,37.0,6.0,32.0,4
4,19.0,3.0,18.0,30.0,0.0,7.0,17.0,5


In [ ]:
full_timetable_data = [item['full_timetable'] for item in data]

combined_schedule_data = {}
for timetable in full_timetable_data:
    combined_schedule_data.update(timetable)
print()

full_timetable_df = pd.DataFrame.from_dict(combined_schedule_data, orient='index')
full_timetable_df.index.name = 'train_id'
full_timetable_df.reset_index(inplace=True)

print("\nDataFrame для полного расписания поездов:")
full_timetable_df.head()


DataFrame для полного расписания поездов:


,train_id,route,free_carriage,timetable
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]","[01:39 - 01:56, 04:06 - 05:10, 13:10 - 14:08, ..."
1,801,"[3, 2, 4, 5]","[35, 37, 40]","[01:00 - 01:49, 09:49 - 11:16, 18:40 - 19:13]"
2,563,"[7, 6, 5]","[19, 7]","[01:16 - 02:28, 03:34 - 04:13, 05:35 - 06:37]"
3,182,"[5, 7]",[36],"[00:42 - 01:27, 04:05 - 04:40]"
4,643,"[1, 3, 6, 5]","[12, 0, 24]","[02:54 - 04:09, 06:19 - 06:52, 10:30 - 11:44, ..."


In [ ]:
full_timetable_data = [item['full_timetable'] for item in data]

combined_schedule_data = {}
for timetable in full_timetable_data:
    combined_schedule_data.update(timetable)
print(combined_schedule_data)

{'853': {'route': ['5', '4', '2', '3', '1'], 'free_carriage': ['11', '30', '33', '33'], 'timetable': ['01:39 - 01:56', '04:06 - 05:10', '13:10 - 14:08', '21:32 - 22:12', '00:22 - 01:44']}, '801': {'route': ['3', '2', '4', '5'], 'free_carriage': ['35', '37', '40'], 'timetable': ['01:00 - 01:49', '09:49 - 11:16', '18:40 - 19:13']}, '563': {'route': ['7', '6', '5'], 'free_carriage': ['19', '7'], 'timetable': ['01:16 - 02:28', '03:34 - 04:13', '05:35 - 06:37']}, '182': {'route': ['5', '7'], 'free_carriage': ['36'], 'timetable': ['00:42 - 01:27', '04:05 - 04:40']}, '643': {'route': ['1', '3', '6', '5'], 'free_carriage': ['12', '0', '24'], 'timetable': ['02:54 - 04:09', '06:19 - 06:52', '10:30 - 11:44', '13:06 - 13:47']}, '743': {'route': ['7', '5', '4'], 'free_carriage': ['37', '26'], 'timetable': ['01:02 - 02:10', '04:48 - 06:11', '08:21 - 09:24']}, '399': {'route': ['5', '4', '2', '3', '1'], 'free_carriage': ['7', '33', '26', '1'], 'timetable': ['02:04 - 02:39', '04:49 - 05:54', '13:54 - 

In [ ]:
stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   station_1       7 non-null      float64 
 1   station_2       7 non-null      float64 
 2   station_3       7 non-null      float64 
 3   station_4       7 non-null      float64 
 4   station_5       7 non-null      float64 
 5   station_6       7 non-null      float64 
 6   station_7       7 non-null      float64 
 7   station_number  7 non-null      category
dtypes: category(1), float64(7)
memory usage: 883.0 bytes


In [ ]:
full_timetable_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3267 entries, 0 to 899
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   train_id       3267 non-null   category
 1   route          3267 non-null   object  
 2   free_carriage  3267 non-null   object  
 3   duration       3267 non-null   int64   
dtypes: category(1), int64(1), object(2)
memory usage: 147.8+ KB


In [ ]:
print(f"Stations dataframe are: {stations_df.shape}\nfull_timetable data frame are: {full_timetable_df.shape}")

Stations dataframe are: (7, 8)
full_timetable data frame are: (900, 4)


In [ ]:
full_timetable_df = full_timetable_df.explode('timetable')

full_timetable_df[['start_time', 'end_time']] = full_timetable_df['timetable'].str.split(' - ', expand=True)

full_timetable_df.drop(columns=['timetable'], inplace=True)

full_timetable_df['start_time'] = pd.to_datetime(full_timetable_df['start_time'], format='%H:%M')
full_timetable_df['end_time'] = pd.to_datetime(full_timetable_df['end_time'], format='%H:%M')

full_timetable_df['duration'] = full_timetable_df['end_time'] - full_timetable_df['start_time']
full_timetable_df['duration'] = full_timetable_df['duration'].astype(str).str.split().str[-1]

full_timetable_df.drop(columns=['start_time', 'end_time'], inplace=True)

print("\nDataFrame после обработки:")
full_timetable_df.head()


DataFrame после обработки:


,train_id,route,free_carriage,duration
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",00:17:00
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",01:04:00
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",00:58:00
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",00:40:00
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",01:22:00


In [ ]:
full_timetable_df['duration'] = full_timetable_df['duration'].apply(convert_duration_to_seconds)

In [ ]:
full_timetable_df.head()

,train_id,route,free_carriage,duration
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",1020
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",3840
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",3480
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",2400
0,853,"[5, 4, 2, 3, 1]","[11, 30, 33, 33]",4920


stations_df - DataFrame с информацией о станциях\
full_timetable_df - DataFrame с информацией о поездах

In [ ]:
full_timetable_df['train_id'] = full_timetable_df['train_id'].astype('category')
stations_df['station_number'] = stations_df['station_number'].astype('category')

In [ ]:
# Это не запускать
user_features = stations_df[['station_id']]

encoder_user = OneHotEncoder(sparse=False)
user_features_encoded = encoder_user.fit_transform(user_features)

user_feature_matrix = user_features_encoded


item_features = full_timetable_df[['train_id']]

encoder_item = OneHotEncoder(sparse=False)
item_features_encoded = encoder_item.fit_transform(item_features)

item_feature_matrix = item_features_encoded

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Используем это
user_ids = stations_df['station_id'].values

item_ids = full_timetable_df['train_id'].values

user_feature_matrix = user_ids.reshape(-1, 1)
item_feature_matrix = item_ids.reshape(-1, 1)

<div class="alert alert-success">
    <h2 align='center'>Model Class</h2>
</div>

In [ ]:
class ContentBasedCollaborativeFiltering(nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super(ContentBasedCollaborativeFiltering, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.item_factors = nn.Embedding(n_items, n_factors)

        self.hidden_layer = nn.Linear(n_factors * 2, 64)
        self.output_layer = nn.Linear(64, 1)

    def forward(self, user, item):
        user_embedding = self.user_factors(user)
        item_embedding = self.item_factors(item)

        input_vector = torch.cat([user_embedding, item_embedding], dim=1)

        hidden_output = torch.relu(self.hidden_layer(input_vector))
        predicted_duration = self.output_layer(hidden_output)

        return predicted_duration

<div class="alert alert-success">
    <h3 align='center'>Creating the DataLoader</h3>
</div>

In [ ]:
class Loader(Dataset):
    def __init__(self, stations_df, full_timetable_df):
        self.stations_df = stations_df
        self.full_timetable_df = full_timetable_df
        self.user_ids = self.stations_df['station_number']
        self.item_ids = self.full_timetable_df['train_id']
        self.duration = self.full_timetable_df['duration']
        self.data = self.build_data()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        user_id = torch.tensor(self.user_ids[idx])
        item_id = torch.tensor(self.item_ids[idx])
        duration = torch.tensor(self.duration[idx])

        return user_id, item_id, duration

    def build_data(self):
        data = []
        for _, row in self.stations_df.iterrows():
            user_id = row['station_number']
            for _, row2 in self.full_timetable_df.iterrows():
                item_id = row2['train_id']
                duration = row2['duration']
                data.append((user_id, item_id, duration))
        return data


In [ ]:
data_loader = Loader(stations_df, full_timetable_df)

In [ ]:
len(data_loader)

22869

In [ ]:
n_users = len(stations_df.station_id.unique())
n_items = len(full_timetable_df.train_id.unique())
n_factors = 20
learning_rate = 0.001
epochs = 10

model = ContentBasedCollaborativeFiltering(n_users, n_items, n_factors=20)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [ ]:
train_size = int(0.8 * len(data_loader))
test_size = len(data_loader) - train_size
train_data, test_data = torch.utils.data.random_split(data_loader, [train_size, test_size])

In [ ]:
batch_size = 4
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
print(len(train_loader))
print(len(test_loader))

4574
1144


In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for user, item, duration in train_loader:
        optimizer.zero_grad()
        output = model(user, item)
        loss = criterion(output, duration)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print("Epoch {}: Loss: {:.4f}".format(epoch+1, total_loss))

    torch.save(model.state_dict(), f"model_weights_epoch_{epoch+1}.pt")

KeyError: ignored

In [ ]:
model.eval()
with torch.no_grad():
    total_precision_at_k = 0.0
    total_recall_at_k = 0.0
    total_map_at_k = 0.0
    k = 5

    for user, item, duration in test_loader:
        predictions = model(user, item)

        recommendations = get_top_k_recommendations(predictions, k)

        precision_at_k = calculate_precision_at_k(duration, recommendations, k)
        recall_at_k = calculate_recall_at_k(duration, recommendations, k)
        map_at_k = calculate_average_precision_at_k(duration, recommendations, k)

        total_precision_at_k += precision_at_k
        total_recall_at_k += recall_at_k
        total_map_at_k += map_at_k

    average_precision_at_k = total_precision_at_k / len(test_loader)
    average_recall_at_k = total_recall_at_k / len(test_loader)
    average_map_at_k = total_map_at_k / len(test_loader)

    print("Average Precision@{}: {:.4f}".format(k, average_precision_at_k))
    print("Average Recall@{}: {:.4f}".format(k, average_recall_at_k))
    print("Average MAP@{}: {:.4f}".format(k, average_map_at_k))

KeyError: ignored

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.4 MB/s eta 0:00:00


In [ ]:
import implicit
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [ ]:
full_timetable_df['route'] = full_timetable_df['route'].apply(lambda x: x[0])
full_timetable_df['free_carriage'] = full_timetable_df['free_carriage'].apply(lambda x: x[0])

interaction_matrix = pd.pivot_table(full_timetable_df, values='duration', index='train_id', columns='route', aggfunc='sum', fill_value=0)

sparse_interaction_matrix = csr_matrix(interaction_matrix.values)

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=200, regularization=0.01, iterations=50)

model.fit(sparse_interaction_matrix)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import pickle

with open("model_weights.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
sparse_interaction_matrix

route,1,2,3,4,5,7
train_id,,,,,,
100,0,0,0,7140,0,0
101,0,0,0,0,0,8820
102,0,0,10560,0,0,0
103,0,0,0,6780,0,0
104,0,7860,0,0,0,0
...,...,...,...,...,...,...
995,0,0,0,0,13980,0
996,0,0,8400,0,0,0
997,0,0,0,6480,0,0


In [ ]:
with open("model_weights.pkl", "rb") as f:
    loaded_model = pickle.load(f)

user_items = csr_matrix(sparse_interaction_matrix[853])

recommended_stations, _ = loaded_model.recommend(0, user_items)
print(recommended_stations)

[3 1 4 0 2 5 0 0 0 0]
